In [1]:
# importing all the libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import copy
import random
import pandas as pd
import math
import scipy
from scipy.stats import norm, expon
from scipy.linalg import null_space
import pandas as pd

import os
cwd = os.getcwd()

%run ../functions_MCMC

In [2]:
# define variables
data_seed = 1223
burnin = 10000 #10k
n_after_burnin = 10000 #10k
delta_t = 0.3

#T=60 gives 200 delta_y
T = 120
n_chains = 4
n_sim = 1

V_F = 2000
V_B = -1500
V = np.array([V_F, V_B])
Lambda = np.array([1, 0.5, 0.3])
log_Lambda = np.log(Lambda)
Pij = np.array([0.2, 0.3, 0.7])
sigma = 50.0
n_param = 9

parameter_names = ['v1', 'v2', 'log(lambda1)', 'log(lambda2)',
                   'log(lambda3)', 'p12', 'p21', 'p31', 'sigma']
parameter_names_tex = [r'$v_1$', r'$v_2$', r'log$(\lambda_1)$',
                       r'log$(\lambda_2)$', r'log$(\lambda_3)$', r'$p_{12}$', r'$p_{21}$', r'$p_{31}$',
                       r'$\sigma$']

#choose initial covariance matrix for resampling
init_cov_matrix = np.array([np.diag(np.array([0.1, 0.1, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]))
                            for _ in range(n_chains)])


correlated = True
up_to_switches = 1
track = True
n_tracks = 4

plots = False
save = True
all_plots = False
plot_posteriors_grid = False
plot_fit = False
plot_fit_median = False

theta_true = list(V) + list(log_Lambda) + list(Pij) + [sigma] #not including values for P for 2x2

In [3]:
#This generates V, Lambda, P, sigma in the n dimensional form
#from the estimated parameter set theta
def get_parameters(theta):
    """Obtaining parameters from theta"""
    V = np.array(list(theta[0:2])+[0.0])
    Lambda = np.exp(theta[2:5])
    P = np.array([[0.0, theta[-4], 1.0-theta[-4]],
                  [theta[-3], 0.0, 1.0-theta[-3]],
                  [theta[-2], 1-theta[-2], 0.0]])
    sigma = 1.0*theta[-1]
    #print(V, Lambda, P, sigma)
    return V, Lambda, P, sigma

def q(theta, cov_matrix=init_cov_matrix):
    """q samples a new theta_star given theta"""
    #print(theta.shape)
    theta_star = np.random.multivariate_normal(theta, cov_matrix)
    while (theta_star[0]<0 or theta_star[0]>2.0*V_F or 
           theta_star[1]>0 or theta_star[1]<2.0*V_B or 
           np.any(theta_star[2:5]<-4) or np.any(theta_star[2:5]>4) or
           np.any(theta_star[5:8]<0) or np.any(theta_star[5:8]>1) or
           theta_star[-1]<0 or np.any(theta_star[-1]>2.0*sigma)):
        theta_star = np.random.multivariate_normal(theta, cov_matrix)
    return theta_star

def theta_init_uniform(seed):
    np.random.seed(seed)
    rand_vec = np.random.uniform(size=n_param)
    theta = ((np.array([2.0*V_F, 2.0*V_B, 8.0, 8.0, 8.0, 1.0, 1.0, 1.0, 2.0*sigma])
              *rand_vec)
              +np.array([0.0, 0.0, -4.0, -4.0, -4.0, 0.0, 0.0, 0.0, 0.0]))
    #print('theta_init =', get_parameters(theta))
    return theta

In [4]:
all_theta, all_log_pi, _, _, dY = distinct_track_runs_MCMC(theta_true = theta_true,
                                 get_parameters = get_parameters,
                                 parameter_names = parameter_names,
                                 parameter_names_tex = parameter_names_tex,
                                 burnin = burnin, n_after_burnin = n_after_burnin,
                                 delta_t = delta_t, T = T,
                                 n_chains = n_chains, n_sim = n_sim,
                                 plots = plots, save = save, all_plots = all_plots,
                                 plot_posteriors_grid = plot_posteriors_grid,
                                 plot_fit = plot_fit,
                                 plot_fit_median = plot_fit_median, track = track,
                                 up_to_switches = up_to_switches,
                                 init_cov_matrix = init_cov_matrix, q = q,
                                 delta_Y = None,
                                 theta_init = None,
                                 theta_init_distribution = theta_init_uniform,
                                 correlated = correlated, n_tracks = n_tracks)

Iteration 0
Running the burnin and adaptively computing the covariance matrices for each chain.


/home/ceccarelli/Documents/MODEL CALIBRATION MCMC/functions_MCMC.py:696: RuntimeWarning: invalid value encountered in divide
  ret = sum([P_delta_y_i_and_st_ip1_giv_st_i(i-1, st, st_im1)
/home/ceccarelli/Documents/MODEL CALIBRATION MCMC/functions_MCMC.py:739: RuntimeWarning: divide by zero encountered in log
  log_like = np.sum(np.log(approx_pdf_theta(theta, get_parameters, delta_t, delta_y,
/home/ceccarelli/Documents/MODEL CALIBRATION MCMC/functions_MCMC.py:846: RuntimeWarning: invalid value encountered in scalar subtract
  log_alpha = min(0.0, cur_log_pi[ind] - prev_log_pi[ind])


Computed the covariance matrices.
Running MCMC with 4 chains
done
R_hat = [1.01390729 1.07275512 1.08442843 1.03002528 1.17285572 1.05301842
 1.08645108 1.01116021 1.00300591]
Iteration 0
Running the burnin and adaptively computing the covariance matrices for each chain.
Computed the covariance matrices.
Running MCMC with 4 chains
done
R_hat = [1.00193103 1.01293102 1.0017331  1.08337951 1.01867149 1.02371252
 1.0112069  1.03801344 1.01141389]
Done simulation 0
Not converged: 1
